In [1]:
!git clone https://github.com/haoweiwang0/Coreset_Prioritization.git
%cd Coreset_Prioritization

Cloning into 'Coreset_Prioritization'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 22 (delta 6), reused 12 (delta 4), pack-reused 0
Receiving objects: 100% (22/22), 578.29 KiB | 1.88 MiB/s, done.
Resolving deltas: 100% (6/6), done.
/content/Coreset_Prioritization


In [2]:
!pip install neural-tangents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.7/248.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 5.4 MB/s eta 0:00:00


In [3]:
!pip install -U "jax[cuda12]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.6/81.6 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.3/363.3 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.5/22.5 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 895.7/895.7 kB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 574.6/574.6 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.5/192.5 MB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.6/217.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [16]:
import bilevel_coreset
import loss_utils
import ntk_generator

import numpy as np
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import torch.nn.functional as F

subset_size = 50

def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
same_seeds(0)

transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

train_dataset = datasets.CIFAR10(root='data', train=True, download=True, transform=transform_test)
test_dataset = datasets.CIFAR10(root='data', train=False, download=False, transform=transform_test)

Files already downloaded and verified


In [17]:
from tqdm.auto import tqdm
import models

def train_model(model, loader):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=5e-4, weight_decay=0)
    criterion = torch.nn.CrossEntropyLoss()
    pbar = tqdm(range(nr_epochs), desc="Training", unit="epoch")

    for epoch in pbar:
        model.train()
        training_loss = 0.0

        for batch_idx, (data, target) in enumerate(loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            training_loss += loss.item()
            optimizer.step()

        pbar.set_postfix({'loss': training_loss / (batch_idx + 1)})

def test_model(model, loader):
    model.to(device)
    criterion = torch.nn.CrossEntropyLoss()

    model.eval()
    pbar = tqdm(loader, desc="Testing", unit="batch")
    correct = 0
    testing_loss = 0.0

    with torch.no_grad():
        for data, target in pbar:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            testing_loss += loss.item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            pbar.set_postfix({'loss': testing_loss / len(loader)})
    test_acc = 1. * correct / len(loader.dataset)
    return test_acc

In [6]:
nr_classes = 10
batch_size = 128
device = 'cuda' if torch.cuda.is_available() else 'cpu'
nr_epochs = 30

resnet18 = models.ResNet18(3).to(device)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

train_model(resnet18, train_loader)
print('Original ResNet - Train accuracy', test_model(resnet18, train_loader))
print('Original ResNet - Test accuracy', test_model(resnet18, test_loader))

Training:   0%|          | 0/30 [00:00<?, ?epoch/s]

Testing:   0%|          | 0/391 [00:00<?, ?batch/s]

Original ResNet - Train accuracy 0.94114


Testing:   0%|          | 0/79 [00:00<?, ?batch/s]

Original ResNet - Test accuracy 0.8728


In [19]:
proxy_kernel_fn = lambda x, y: ntk_generator.generate_resnet_ntk(x.transpose(0, 2, 3, 1), y.transpose(0, 2, 3, 1), skip=20)

In [20]:
# work only on the first 10000 samples for speedup

limit = 10000

loader = torch.utils.data.DataLoader(train_dataset, batch_size=len(train_dataset), shuffle=False)
X, y = next(iter(loader))
X, y = X.numpy(), y.numpy()
all_inds = np.random.choice(np.arange(len(train_dataset)), 10000, replace=False)
X = X[all_inds]
y = y[all_inds]

bc = bilevel_coreset.BilevelCoreset(outer_loss_fn=loss_utils.cross_entropy,
                                    inner_loss_fn=loss_utils.cross_entropy, out_dim=10,
                                    max_outer_it=10, outer_lr=0.05, max_inner_it=200)
coreset_inds, _ = bc.build_with_representer_proxy_batch(X, y, subset_size, proxy_kernel_fn, cache_kernel=True,
                                                        start_size=10, inner_reg=1e-7)

Coreset size 10, outer_loss 5.48, inner loss 0.000538
Coreset size 20, outer_loss 2.84, inner loss 0.000966
Coreset size 30, outer_loss 2.55, inner loss 0.00131
Coreset size 40, outer_loss 2.3, inner loss 0.00166
Coreset size 50, outer_loss 2.2, inner loss 0.00199


In [21]:
nr_epochs = 1000

coreset_net = models.ResNet18(3).to(device)
coreset_subset = Subset(train_dataset, coreset_inds)
train_loader = torch.utils.data.DataLoader(coreset_subset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

train_model(coreset_net, train_loader)
print('Coreset ResNet - Train accuracy', test_model(coreset_net, train_loader))
print('Coreset ResNet - Test accuracy', test_model(coreset_net, test_loader))

Training:   0%|          | 0/1000 [00:00<?, ?epoch/s]

Testing:   0%|          | 0/1 [00:00<?, ?batch/s]

Coreset ResNet - Train accuracy 1.0


Testing:   0%|          | 0/79 [00:00<?, ?batch/s]

Coreset ResNet - Test accuracy 0.2348


In [22]:
def test_with_logits(model, coreset_model, loader):
    model.eval()
    logits = []
    with torch.no_grad():
        for data, target in loader:
            data, target = data.to(device), target.to(device)
            output = F.softmax(model(data), dim=1)
            coreset_output = F.softmax(coreset_model(data), dim=1)
            logits.append((data, output, coreset_output, target))

    return logits

logits = test_with_logits(resnet18, coreset_net, test_loader)

In [23]:
prioritized_test_data = []

# calculate the similarity score between the logits from
# original DNN and coreset-trained DNN
# reorder test cases based on the similarity score
def test_case_prioritization(logits):
    result = []
    probs = torch.cat([item[1] for item in logits], dim=0).cpu().numpy()
    coreset_probs = torch.cat([item[2] for item in logits], dim=0).cpu().numpy()
    inputs = torch.cat([item[0] for item in logits], dim=0).cpu().numpy()
    targets = torch.cat([item[3] for item in logits], dim=0).cpu().numpy()
    for i in range(probs.shape[0]):
        item = probs[i]
        coreset_item = coreset_probs[i]
        similarity_score = F.cosine_similarity(torch.tensor(item).unsqueeze(0), torch.tensor(coreset_item).unsqueeze(0))
        result.append({'similarity_score': similarity_score,
                       'probabilities': item,
                       'input': inputs[i],
                       'target': targets[i]})
    result = sorted(result, key=lambda x: x['similarity_score'], reverse=False)
    return result

In [24]:
prioritized_test_data = test_case_prioritization(logits)

In [25]:
from torch.utils.data import TensorDataset

inputs = torch.stack([torch.from_numpy(item['input']) for item in prioritized_test_data])
targets = torch.stack([torch.tensor(item['target']) for item in prioritized_test_data])
tensor_dataset = TensorDataset(inputs, targets)
prioritized_loader = torch.utils.data.DataLoader(tensor_dataset, batch_size=1, shuffle=False)

In [26]:
def test_with_APFD(model, loader):
    model.to(device)
    model.eval()

    TFs = 10 * [0]

    with torch.no_grad():
        for ids, (data, target) in enumerate(loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            category = output.argmax(dim=1, keepdim=True)
            if category != target and TFs[target] == 0:
                TFs[target] = ids + 1

            if all(TFs):
                break

    APFD = 1 - (sum(TFs) / (10 * len(loader))) + 1 / (2 * len(loader))

    return APFD

print('ResNet - APFD', test_with_APFD(resnet18, prioritized_loader))

ResNet - APFD 0.9031
